In [1]:
using Plots

In [2]:
include("everest_option.jl")

everest_option_monte_carlo (generic function with 2 methods)

In [11]:
function compare_variance(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
                        S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64}, correlation_matrix::Array{Float64})


    normal = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix,"basic")
    antithetic = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix,"antithetic")
    quasi = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix,"quasi_monte_carlo")
    moment_matching = price_everest_moment_matching(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix)
    LHS = everest_option_monte_carlo(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix,"LHS")

        
    mean_elements = normal[1]
    bottom_elements = normal[2]
    upper_elements = normal[3]

    mean_antithetic = antithetic[1]
    bottom_antithetic = antithetic[2]
    upper_antithetic = antithetic[3]

    mean_quasi = quasi

    mean_moment_matching = moment_matching[1]
    bottom_moment_matching = moment_matching[2]
    upper_moment_matching = moment_matching[3]

    mean_LHS = LHS[1]
    bottom_LHS = LHS[2]
    upper_LHS = LHS[3]

    length_normal = upper_elements-bottom_elements
    length_antithetic = upper_antithetic - bottom_antithetic
    length_moment_matching = upper_moment_matching - bottom_moment_matching
    length_LHS = upper_LHS-bottom_LHS

    return [mean_elements, mean_antithetic, mean_moment_matching, mean_LHS, mean_quasi, 
            length_normal, length_antithetic, length_moment_matching, length_LHS]
end


function num_of_sims(num_of_sims::Array{Int}, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
    S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64}, correlation_matrix::Array{Float64})

    results::Matrix{Float64} = zeros(length(num_of_sims), 9)
    
    for (ind, num_of_sim) in enumerate(num_of_sims)
        results[ind,:] = compare_variance(num_of_sim, α, T, N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix)
    end

    return results

end 

function mu_value(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
                S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64}, correlation_matrix::Array{Float64})


    results::Matrix{Float64} = zeros(length(mu), 9)
    for (ind, m) in enumerate(mu)
        results[ind,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀, [m for i in 1:basket_volume], sigma, correlation_matrix)
    end

    return results

end

function interest_rate(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Array{Float64}, C::Float64, basket_volume::Int, 
                    S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64}, correlation_matrix::Array{Float64})


    results::Matrix{Float64} = zeros(length(rs), 9)
    for (ind, r) in enumerate(rs)
    results[ind,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀, mu, sigma, correlation_matrix)
    end

    return results

end 


function maturity(num_of_sim::Int, α::Float64,T::Int, r::Float64, C::Float64, basket_volume::Int, 
                    S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64}, correlation_matrix::Array{Float64})

    results::Matrix{Float64} = zeros(T, 9)
    for maturity in 1:T
        results[maturity,:] = compare_variance(num_of_sim, α, maturity, maturity*250, r, C, basket_volume, S₀, mu, sigma, correlation_matrix)
    end

    return results

end


function correlation(num_of_sim::Int, α::Float64,T::Int,N::Int, r::Float64, C::Float64, basket_volume::Int, 
    S₀::Array{Float64}, mu::Array{Float64}, sigma::Array{Float64})

    corr_matrices::Vector{Matrix{Float64}} = [[1.0 0.1 0.1; 0.1 1.0 0.1; 0.1 0.1 1.0], [1.0 0.5 0.4; 0.5 1.0 0.6; 0.4 0.6 1.0], [1.0 0.9 0.8; 0.9 1.0 0.7; 0.8 0.7 1.0]] 

    if basket_volume == 7
    corr_matrices = [[
    1.0  0.1  0.1  0.1  0.1  0.1  0.1;
    0.1  1.0  0.1  0.1  0.1  0.1  0.1;
    0.1  0.1  1.0  0.1  0.1  0.1  0.1;
    0.1  0.1  0.1  1.0  0.1  0.1  0.1;
    0.1  0.1  0.1  0.1  1.0  0.1  0.1;
    0.1  0.1  0.1  0.1  0.1  1.0  0.1;
    0.1  0.1  0.1  0.1  0.1  0.1  1.0
    ],
    [
    1.0  0.5  0.4  0.3  0.2  0.1  0.1;
    0.5  1.0  0.6  0.5  0.4  0.2  0.1;
    0.4  0.6  1.0  0.4  0.3  0.1  0.1;
    0.3  0.5  0.4  1.0  0.6  0.2  0.1;
    0.2  0.4  0.3  0.6  1.0  0.5  0.1;
    0.1  0.2  0.1  0.2  0.5  1.0  0.4;
    0.1  0.1  0.1  0.1  0.1  0.4  1.0
    ],
    [
    1.0  0.9  0.8  0.7  0.6  0.5  0.4;
    0.9  1.0  0.9  0.8  0.7  0.6  0.5;
    0.8  0.9  1.0  0.7  0.6  0.5  0.4;
    0.7  0.8  0.7  1.0  0.9  0.4  0.3;
    0.6  0.7  0.6  0.9  1.0  0.3  0.2;
    0.5  0.6  0.5  0.4  0.3  1.0  0.1;
    0.4  0.5  0.4  0.3  0.2  0.1  1.0
    ]] 
    end

    results::Matrix{Float64} = zeros(3, 9)
    for correlation in eachindex(corr_matrices)
        results[correlation,:] = compare_variance(num_of_sim, α,T,N, r, C, basket_volume, S₀, mu, sigma, corr_matrices[correlation])
    end
    return results

end


function plot_variance(vals, variance, title, xlabel)
    scatter(vals,variance, label = ["normal" "antithetic" "moment_matching" "LHS"], title = title, legend = :outertop)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", title, "_everest_", ".svg"))
end

function plot_mean(vals, mean, title, xlabel)
    scatter(vals, mean, label=["mean_elements" "mean_antithetic" "mean_moment_matching" "mean_LHS" "mean_quasi"], legend = :outertop)
    title!(title)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", title,"_everest_", ".svg"))
end

function plot_bar_plot(results,name)
    bar(["słaba", "średnia", "silna"],results,label=name,title=string("długość przedziału ~ korelacja dla metody ",name), xlabel = "siła korelacji między spółkami", xticks = :all,legend=false, fmt = :svg)    
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\everest\\", name, "_everest_", ".svg"))
end

plot_bar_plot (generic function with 1 method)

In [4]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 

3×3 Matrix{Float64}:
 1.0  0.3  0.4
 0.3  1.0  0.1
 0.4  0.1  1.0

# Liczba symulacji a szerokość przedziału

In [9]:
n_of_sims = [100:10:2000;]
sims::Matrix{Float64} = num_of_sims(n_of_sims,0.01,1,250,0.03,8.0,3,[10.0,10.0,10.0],[0.05,0.05,0.05],[0.15,0.15,0.15],cov_matrix)
variance = sims[:,6:end]
plot_variance(n_of_sims,variance,"długość przedziału ufności ~ wielkość próby", "wielkość próby")
means = sims[:,1:5]
plot_mean(n_of_sims,means,"cena opcji ~ wielkość próby", "wielkość próby")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\cena opcji ~ wielkość próby_everest_.svg"

# Variance ~ trend

In [17]:
mu = [0.01:0.001:0.1;]
sims_mu::Matrix{Float64} = mu_value(1000,0.01,1,250,0.03,8.0,3,[10.0,10.0,10.0],mu,[0.15,0.15,0.15],cov_matrix)
variance_mu = sims_mu[:,6:end]
plot_variance(mu,variance_mu, "długość przedziału ufności ~ trend", "trend")
means_mu = sims_mu[:,1:5]
plot_mean(mu,means_mu, "cena opcji ~ trend", "trend")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\cena opcji ~ trend_everest_.svg"

# Stopy procentowe

In [22]:
rs = [0.01:0.001:0.1;]
sims_rs::Matrix{Float64} = interest_rate(1000,0.01,1,250,rs,8.0,3,[10.0,10.0,10.0],[0.05,0.05,0.05],[0.15,0.15,0.15],cov_matrix)
variance_rs = sims_rs[:,6:end]
plot_variance(rs,variance_rs, "długość przedziału ufności ~ stopy procentowe", "stopy procentowe")
means_rs = sims_rs[:,1:5]
plot_mean(rs,means_rs, "cena opcji ~ stopy procentowe", "stopy procentowe")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\cena opcji ~ stopy procentowe_everest_.svg"

# Matura

In [6]:
sims_maturity::Matrix{Float64} = maturity(1000,0.01,10,0.03,8.0,3,[10.0,10.0,10.0],[0.05,0.05,0.05],[0.15,0.15,0.15],cov_matrix)
mat= [i for i in 1:10]
variance_maturity = sims_maturity[:,6:end]
plot_variance(mat,variance_maturity, "długość przedziału ~ matura", "matura")
means_maturity = sims_maturity[:,1:5]
plot_mean(mat,means_maturity, "cena opcji ~ matura", "matura")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\cena opcji ~ matura_everest_.svg"

# Korelacja

In [10]:
sims_correlation::Matrix{Float64} = correlation(1000,0.01,1,250,0.03,8.0,3,[10.0,10.0,10.0],[0.05,0.05,0.05],[0.15,0.15,0.15])
plot_bar_plot(sims_correlation[:,6],"basic")
plot_bar_plot(sims_correlation[:,7],"antithetic")
plot_bar_plot(sims_correlation[:,8],"moment_matching")
plot_bar_plot(sims_correlation[:,9],"LHS")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\LHS_everest_.svg"